# Preproccessing script for CMIP6 potential temperature, salinity, and DIC fields

All preprocessing is to be done on the *native* CMIP6 output fields, then included when calculating depth-integrated temperature, salinity, and DIC

Our preprocessing includes the following:
1. Calculation of annual averages using momlevel (takes into account irregular month lengths) -- saving this to a zarr directory
2. Calculation of linear drifts from preindustrial control (piControl) runs. The use of a linear drift rather than a higher-order drift is in line with how XMIP and pangeo operate (double-check this)
3. Regridding CMIP fields to a standard 1x1 (or 1.25x1.25) grid (1x1 for theoretical work, 1.25x1.25 for work with Argo data)
3. Integrating CMIP fields to various depths (100m, 500m, 1000m, 2000m)
4. Concatenating the integrated CMIP fields into a super-timeseries

In [1]:
import xarray as xr
import momlevel as ml
import numpy as np

In [2]:
from dask.distributed import Client
client = Client('tcp://127.0.0.1:8786')

In [3]:
client

<Client: 'tcp://192.168.0.4:8786' processes=4 threads=8, memory=14.90 GiB>

In [4]:
#path = "/Users/keturner/Downloads"
path = "/Volumes/KT-TOSHIBA/ENOI/CMIP6/preindustrial_controls"
var = "dissic"
model = "ACCESS"

In [ ]:
#years = [196, 200, 205, 210, 215]

In [ ]:
#for i in np.arange(len(years)):
#    ds = xr.open_mfdataset(f"{path}/{var}_Omon_{model}*{years[i]}*.nc", use_cftime=True,
#                          chunks={"time":12, "lev":25, "j":330, "i":120})
#    so_ann = ml.util.annual_average(ds.so)
#    so_ann = so_ann.astype("float32").to_dataset(name="so_annual")
#    so_ann.chunk({"time":20}).to_netcdf(f"ann_{var}_{model}{i}.nc")

In [5]:
ds = xr.open_mfdataset(f"{path}/{var}_Omon_{model}*.nc",
                      use_cftime=True,
                      chunks={"time":120, "lev":10, "j":50, "i":360})
ds.dissic

<xarray.DataArray 'dissic' (time: 2400, lev: 50, j: 300, i: 360)>
dask.array<concatenate, shape=(2400, 50, 300, 360), dtype=float32, chunksize=(120, 10, 50, 360), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) object 0101-01-16 12:00:00 ... 0300-12-16 12:00:00
  * lev        (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j          (j) int32 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
    longitude  (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
Attributes:
    standard_name:   mole_concentration_of_dissolved_inorganic_carbon_in_sea_...
    long_name:       Dissolved Inorganic Carbon Concentration
    comment:         Dissolved inorganic carbon (CO3+HCO3+H2CO3) concentration
    units:           mol m-3
    original_units:  mmol m-3
    history:         2019-11-13T00:34:45Z altered by CMOR: Converted units fr...
    cell_methods:    area: mean where sea time: mean
    cell_measures:   area: areacello volume: volcello

In [8]:
dissic_ann = ml.util.annual_average(ds.dissic)
dissic_ann = dissic_ann.astype("float32").to_dataset(name="dissic_annual")

In [9]:
dissic_ann.dissic_annual

<xarray.DataArray 'dissic_annual' (time: 200, lev: 50, j: 300, i: 360)>
dask.array<astype, shape=(200, 50, 300, 360), dtype=float32, chunksize=(1, 10, 50, 360), chunktype=numpy.ndarray>
Coordinates:
  * lev        (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
  * j          (j) int32 0 1 2 3 4 5 6 7 8 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 0 1 2 3 4 5 6 7 8 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
    longitude  (j, i) float64 dask.array<chunksize=(50, 360), meta=np.ndarray>
  * time       (time) object 0101-07-02 12:00:00 ... 0300-07-02 12:00:00
Attributes:
    standard_name:   mole_concentration_of_dissolved_inorganic_carbon_in_sea_...
    long_name:       Dissolved Inorganic Carbon Concentration
    comment:         Dissolved inorganic carbon (CO3+HCO3+H2CO3) concentration
    units:           mol m-3
    original_units:  mmol m-3
    history:         2019-11-13T00:34:45Z altered by CMOR: Converted units fr...
    cell_methods:    area: mean where sea time: mean
    cell_measures:   area: areacello volume: volcello

In [10]:
%%time
dissic_ann.chunk({"time":20}).to_zarr(f"ann_{var}_{model}")

CPU times: user 10.7 s, sys: 717 ms, total: 11.4 s
Wall time: 1h 28s


In [ ]:
dic_ann.close()
ds.close()

In [ ]:
test = xr.open_mfdataset("ann_so_UKESM*", use_cftime=True)

In [ ]:
test.dt.year